<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания  10


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Создать базовый класс Invoice в C#, который будет представлять информацию о
фактурах за поставленные товары или оказанные услуги. На основе этого класса
разработать 2-3 производных класса, демонстрирующих принципы наследования и
полиморфизма. В каждом из классов должны быть реализованы новые атрибуты и
методы, а также переопределены некоторые методы базового класса для
демонстрации полиморфизма.

Требования к базовому классу Invoice:

• Атрибуты: Номер фактуры (InvoiceNumber), Дата выдачи (IssueDate), Общая
сумма (TotalAmount).

• Методы:
1. CalculateTotal(): метод для расчета общей суммы по фактуре.
2. AddLine(LineItem lineItem): метод для добавления позиции в фактуру.
3. RemoveLine(LineItem lineItem): метод для удаления позиции из
фактуры.

Требования к производным классам:
1. ТоварнаяФактура (GoodsInvoice): Должна содержать дополнительные
атрибуты, такие как Дата поставки (SupplyDate). Метод AddLine() должен
быть переопределен для добавления информации о дате поставки товара
при добавлении позиции.
2. УслуговаяФактура (ServiceInvoice): Должна содержать дополнительные
атрибуты, такие как Дата оказания услуги (ServiceDate).
Метод RemoveLine() должен быть переопределен для добавления
информации о причине аннулирования услуги при удалении позиции.
3. КомбинированнаяФактура (CombinedInvoice) (если требуется третий класс):
Должна содержать дополнительные атрибуты, такие как Наличие возврата
(ReturnAllowed). Метод CalculateTotal() должен быть переопределен для
учета возможного возврата товара или услуги при расчете общей суммы.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [2]:
using System;
using System.Collections.Generic; 

/*
Это стандартная библиотека в C#, которая содержит коллекции данных (контейнеры) для работы с элементами разных типов. Основные коллекции, которые предоставляет это пространство имен, — это обобщенные (generic) коллекции, такие как:

List<T> — динамический список, который может хранить элементы любого типа T. Например: List<int>, List<string>.
Dictionary<TKey, TValue> — коллекция пар "ключ-значение".
Queue<T> — очередь (FIFO — first in, first out, "первый вошел, первый вышел").
Stack<T> — стек (LIFO — last in, first out, "последний вошел, первый вышел").----------------------------------------------------------------
*/

class LineItem
{
    public string Description { get; set; }   // Описание товара или услуги в фактуре
    public decimal Price { get; set; }        // Цена товара или услуги за 1 штуку
    public int Quantity { get; set; }         // Количество товара в фактуре
    public bool ReturnAllowed { get; set; }    // Можно ли вернуть товар/услугу обратно
    public int ReturnedQuantity { get; set; }  // Количество возвращенных единиц из фактуры

    // Конструктор
    public LineItem(string description, decimal price, int quantity, bool returnAllowed)
    {
        Description = description;
        Price = price;
        Quantity = quantity;
        ReturnAllowed = returnAllowed;
        ReturnedQuantity = 0; // Изначально ни один товар не возвращен
    }

    // Метод для получения общей стоимости товаров с учетом количества не возвращенных единиц
    public decimal GetTotal()
    {
        return Price * (Quantity - ReturnedQuantity);
    }

    // Метод для возврата товара (частичный возврат)
    public virtual bool Return(int returnQuantity)
    {
        if (ReturnAllowed && returnQuantity <= (Quantity - ReturnedQuantity))
        {
            ReturnedQuantity += returnQuantity;
            return true;
        }
        return false;
    }
}

// Базовый класс для всех типов фактур
abstract class Invoice
{
    public string InvoiceNumber { get; set; }
    public DateTime IssueDate { get; set; }
    public decimal TotalAmount { get; protected set; }

    protected List<LineItem> lineItems;

    // Конструктор
    public Invoice(string invoiceNumber, DateTime issueDate)
    {
        InvoiceNumber = invoiceNumber;
        IssueDate = issueDate;
        lineItems = new List<LineItem>(); // создает новый экземпляр списка объектов типа LineItem
    }

    public virtual void CalculateTotal()
    {
        TotalAmount = 0;
        foreach (var item in lineItems)
        {
            TotalAmount += item.GetTotal();
        }
    }

    // Метод для добавления позиции
    public virtual void AddLine(LineItem lineItem)
    {   
        lineItems.Add(lineItem);
    }

    // Метод для удаления позиции
    public virtual void RemoveLine(LineItem lineItem)
    {
        lineItems.Remove(lineItem);
        CalculateTotal();
        Console.WriteLine($"Удалена позиция: {lineItem.Description}\nОбщая сумма по комбинированной фактуре после удаления: {TotalAmount}");
    }
}

class GoodsItem : LineItem
{   
    public DateTime SupplyDate{get; set;}

    public GoodsItem(string description, decimal price, int quantity, DateTime supplyDate, bool returnAllowed)
            : base(description, price, quantity, returnAllowed)
    {
        SupplyDate = supplyDate;
    }
}

class ServiceItem : LineItem
{
    public DateTime ServiceDate{get; set;}
        public ServiceItem(string description, decimal price, int quantity, DateTime serviceDate, bool returnAllowed)
            : base(description, price, quantity, returnAllowed)
    {
        ServiceDate = serviceDate;
    }
}

class CombinedInvoice : Invoice
{
    public bool ReturnAllowed { get; set; }
    public int TotalReturnedItems { get; private set; }

    public CombinedInvoice(string invoiceNumber, DateTime issueDate, bool returnAllowed)
        : base(invoiceNumber, issueDate)
    {
        ReturnAllowed = returnAllowed;
    }

    public override void AddLine(LineItem lineItem)
    {
        base.AddLine(lineItem);
        // Проверка типа позиции
        if (lineItem is GoodsItem goodsItem && lineItem.ReturnAllowed == true)
        {
            Console.WriteLine($"Добавлен товар: {goodsItem.Description}, количество: {goodsItem.Quantity}. Дата поставки товара: {goodsItem.SupplyDate:dd.MM.yyyy}. Данный товар можно вернуть.");
        }
        if (lineItem is GoodsItem goodsItemNonReturn  && lineItem.ReturnAllowed == false)
        {
            Console.WriteLine($"Добавлен товар: {goodsItemNonReturn.Description}, количество: {goodsItemNonReturn.Quantity}. Дата поставки товара: {goodsItemNonReturn.SupplyDate:dd.MM.yyyy}. Данный товар нельзя вернуть.");
        }
        if (lineItem is ServiceItem serviceItem  && lineItem.ReturnAllowed == true)
        {
            Console.WriteLine($"Добавлена услуга: {serviceItem.Description}, количество: {serviceItem.Quantity}. Дата оказании услуги: {serviceItem.ServiceDate:dd.MM.yyyy}. Данную услугу можно вернуть.");
        }
        if (lineItem is ServiceItem serviceItemNonReturn  && lineItem.ReturnAllowed == false)
        {
            Console.WriteLine($"Добавлена услуга: {serviceItemNonReturn.Description}, количество: {serviceItemNonReturn.Quantity}. Дата оказании услуги: {serviceItemNonReturn.ServiceDate:dd.MM.yyyy}. Данную услугу нельзя вернуть.");
        }
        CalculateTotal();
    }

    public void ReturnItem(LineItem lineItem, int returnQuantity)
    {
        if (!ReturnAllowed)
        {
            Console.WriteLine("Возврат товара или услуги для данной фактуры запрещен.");
            return;
        }

        if (!lineItem.ReturnAllowed)
        {
            Console.WriteLine($"Возврат для товара/услуги {lineItem.Description} запрещен.");
            return;
        }

        if (lineItem.Return(returnQuantity))
        {
            Console.WriteLine($"Возвращено количество: {returnQuantity} шт. из позиции: {lineItem.Description}. Остаток позиции в фактуре: {lineItem.Quantity - lineItem.ReturnedQuantity}.");
            TotalReturnedItems += returnQuantity;
            CalculateTotal();
            Console.WriteLine($"Общая сумма по комбинированной фактуре после возврата: {TotalAmount}.");
        }
        else
        {
            Console.WriteLine($"Невозможно вернуть {returnQuantity} шт. из {lineItem.Description}. Возможно, запрашиваемое количество больше, чем доступно для возврата или же {lineItem.Description} не находится в фактуре.");
        }

        // Пересчитываем общую сумму после возврата
        CalculateTotal();
    }

}

// Создаем комбинированную фактуру с разрешенным возвратом
CombinedInvoice combinedInvoice = new CombinedInvoice("INV003", DateTime.Now, true);

// Добавляем товары
GoodsItem item1 = new GoodsItem("Товар 1", 100, 2, DateTime.Now.AddDays(7), true);  // Данные о товаре в фактуре
GoodsItem item2 = new GoodsItem("Товар 2", 200, 3, DateTime.Now.AddDays(10), false); // Данные о товаре в фактуре



combinedInvoice.AddLine(item1);
combinedInvoice.AddLine(item2);

// Добавляем услуги
ServiceItem service1 = new ServiceItem("Услуга 1", 300, 0, DateTime.Now.AddDays(-2), true); // Данные об услуге в фактуре 
ServiceItem service2 = new ServiceItem("Услуга 2", 400, 1, DateTime.Now.AddDays(1), false); // Данные об услуге в фактуре

combinedInvoice.AddLine(service1);
combinedInvoice.AddLine(service2);

// combinedInvoice.RemoveLine(item1);

// Рассчитываем общую сумму фактуры
combinedInvoice.CalculateTotal();
Console.WriteLine($"Общая сумма фактуры: {combinedInvoice.TotalAmount}");
Console.WriteLine("\n\n\nПРОДАЖА \n\n\n");

// // Возвращаем часть товара из фактуры
combinedInvoice.ReturnItem(item1, 1);  // Возвращаем 2 шт. из Товар 1
combinedInvoice.ReturnItem(service1, 1);  // Возвращаем 2 шт. из Товар 1
combinedInvoice.ReturnItem(item1, 1);  // Возвращаем 2 шт. из Товар 1
combinedInvoice.ReturnItem(item1, 1);  // Возвращаем 2 шт. из Товар 1
// Возвращаем услугу полностью
// combinedInvoice.ReturnItem(service1, 1);  // Возвращаем услугу

// combinedInvoice.CheckItemAvailability(service1);

// Рассчитываем общую сумму после возврата
combinedInvoice.CalculateTotal();


Console.WriteLine($"Общее количество возвращенных товаров и услуг: {combinedInvoice.TotalReturnedItems}.");
